# Image data preprocessing
https://keras.io/api/preprocessing/image/

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
import os
# import random
import cv2
import sys

sys.path.insert(1, '../src')
import app_constants as const

#Model
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img

In [ ]:
# https://keras.io/api/preprocessing/image/#imagedatagenerator-class

datagen = ImageDataGenerator(rotation_range=5, 
                            width_shift_range=0.1, 
                            height_shift_range=0.1,
                            shear_range=0.15,
                            # fill_mode = 'nearest',
                            horizontal_flip=True)

In [ ]:
for img in os.listdir(const.PNEUMONIA_DIR):
    img_path = os.path.join(const.PNEUMONIA_DIR,img)
    img = load_img(img_path)  
    x = img_to_array(img) 
    # Reshape the input image 
    x = x.reshape((1, ) + x.shape)
    i = 0

    # generate 5 new augmented images 
    for batch in datagen.flow(x, batch_size = 1, 
                      save_to_dir = const.PNEUMONIA_AUG_DIR,  
                      save_prefix ='aug', save_format ='png'):
        i += 1
        if i > 5: 
            break

In [ ]:
xrays_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(const.DATA_DIR, '*','*.png'))}

xrays_df = pd.DataFrame.from_dict(xrays_dict, orient = 'index').reset_index()
xrays_df.columns = ['id', 'path']
labels = xrays_df.id.str.split('-').str[0]
xrays_df['label'] = labels
xrays_df["category"] = xrays_df["label"].map(const.CATEGORY.get) 
print(xrays_df.shape)

In [ ]:
xrays_df['image'] = xrays_df['path'].map(lambda x: np.asarray(Image.open(x).resize((const.IMAGE_SIZE,const.IMAGE_SIZE))))

In [ ]:
xrays_df['label'].value_counts().plot.bar(color=["green","red", "orange"])